In [ ]:
# import the libraries
#import numpy as np
#import pandas as pd
#import mmlspark

#import subprocess
#import sys
#import os
#import re
#import time
#import atexit
import matplotlib.pyplot as plt

#from pyspark.sql import Row
#from pyspark.sql.functions import col, unix_timestamp, round

#from pyspark import SparkConf
#from pyspark import SparkContext
#from pyspark import SQLContext
#import pyspark.sql.functions as F
#from pyspark.sql.functions import concat, col, udf, lag, date_add, explode, lit, unix_timestamp
#from pyspark.sql.functions import month, weekofyear, dayofmonth
#from pyspark.sql.functions import datediff, to_date, lit, unix_timestamp
# from pyspark.sql.types import *
#from pyspark.sql.types import DateType
# from pyspark.sql.dataframe import *
#from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml.classification import *
#from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
#from pyspark.ml.feature import StandardScaler, PCA, RFormula
from pyspark.ml import Pipeline, PipelineModel
#from pyspark.mllib.stat import Statistics
#from pyspark.sql.functions import UserDefinedFunction
#from pyspark.sql.functions import *

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, VectorAssembler, RFormula
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from sklearn.metrics import roc_curve,auc
#from pyspark.sql.functions import month, weekofyear, dayofmonth
#from pyspark.ml.feature import ChiSqSelector
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import StandardScaler
#from pyspark.ml.feature import PCA
#from pyspark.ml.feature import MinMaxScaler
#from pyspark.sql.types import DoubleType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# load the previous created final dataset into the workspace
from azure.storage.blob import BlockBlobService
import glob
import os

# define parameters 
ACCOUNT_NAME = "pdmvienna"
ACCOUNT_KEY = "PDuXK61GpmMVWMrWdvr29THbPdlOXa61fN5RfgQV/jBO8berC1zLzZ678Nxrx+D3CRp4+ZvSff9al+lrUh8qUQ=="
CONTAINER_NAME = "featureengineering"

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)


In [ ]:

# create a local path where to store the results later.
LOCAL_DIRECT = 'model_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'featureengineering_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

data = spark.read.parquet('model_result.parquet')
#data.persist()
data.show(10)
print('Feature engineering final dataset files loaded!')

# Prepare the train/test data

In [ ]:
# define list of input columns for downstream modeling - note model variable was removed as string was not supported
input_features = [
'volt_rollingmean_3',
'rotate_rollingmean_3',
'pressure_rollingmean_3',
'vibration_rollingmean_3',
'volt_rollingmean_24',
'rotate_rollingmean_24',
'pressure_rollingmean_24',
'vibration_rollingmean_24',
'volt_rollingstd_3',
'rotate_rollingstd_3',
'pressure_rollingstd_3',
'vibration_rollingstd_3',
'volt_rollingstd_24',
'rotate_rollingstd_24',
'pressure_rollingstd_24',
'vibration_rollingstd_24',
'error1sum_rollingmean_24',
'error2sum_rollingmean_24',
'error3sum_rollingmean_24',
'error4sum_rollingmean_24',
'error5sum_rollingmean_24',
'comp1sum',
'comp2sum',
'comp3sum',
'comp4sum',
'age',
'model_encoded'    
]

label_var = ['label_e']
key_cols =['machineID','dt_truncated']


In [ ]:
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
data = va.transform(data).select('machineID','dt_truncated','label_e','features')

In [ ]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(data)

In [ ]:
# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(data)

In [ ]:
# split the data into train/test based on date
training = data.filter(data.dt_truncated > "2015-01-01").filter(data.dt_truncated < "2015-09-30")
testing = data.filter(data.dt_truncated > "2015-09-30")

print(training.count())
print(testing.count())

# Classification models

## Random Forest Classifier

In [ ]:
# train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# chain indexers and forest in a Pipeline
pipeline_rf = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# train model.  This also runs the indexers.
model_rf = pipeline_rf.fit(training)

In [ ]:
# make predictions.
predictions_rf = model_rf.transform(testing)
predictions_rf.groupby('indexedLabel', 'prediction').count().show()

In [ ]:
predictions_rf.dtypes

In [ ]:
predictionAndLabels = predictions_rf.select("indexedLabel", "prediction").rdd

In [ ]:
# select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "accuracy"}))

In [ ]:
print("Weighted Precision = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedPrecision"}))

In [ ]:
print("Weighted Recall = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedRecall"}))

In [ ]:
print("F1 = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "f1"}))

# Decision Tree Classifier

In [ ]:
# train a DT model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# chain indexers and forest in a Pipeline
pipeline_dt = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# train model.  This also runs the indexers.
model_dt = pipeline_dt.fit(training)

In [ ]:
# make predictions.
predictions_dt = model_dt.transform(testing)
predictions_dt.groupby('indexedLabel', 'prediction').count().show()

In [ ]:
predictions_dt.dtypes

In [ ]:
predictionAndLabels = predictions_dt.select("indexedLabel", "prediction").rdd

In [ ]:
# select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "accuracy"}))

In [ ]:
print("Weighted Precision = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedPrecision"}))

In [ ]:
print("Weighted Recall = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedRecall"}))

In [ ]:
print("F1 = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "f1"}))

In [ ]:
# write the final model as parquet file in blob location 
# https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md

# you decide to partition the dataframe into three files and save them in the current folder.
# if you wish to visualize them in the run history Output Files, specify the path as 
# './outputs/multiple_files.parquet'.
#model_dt.write.mode('overwrite').parquet('model.parquet')
model_dt.save('model.parquet')

# unlike the single file case, for multiple files we need to first delete results 
# from the previous run before uploading.
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'model.parquet' in blob.name:
        my_service.delete_blob(CONTAINER_NAME, blob.name)

print("Model files saved!")